# Урок 3.  Embedding word2vec fasttext

## Домашнее задание

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

### Подключение библиотек

In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

### Загрузка и подготовка данных

In [6]:
df = pd.read_csv('/content/drive/MyDrive/gb_NLP/lesson-3/gazeta.csv', usecols=['text', 'title'])
df.head()

,title,text
0,Госдума сокращает срок действия ставки экспорт...,Госдума приняла сегодня в первом чтении и сраз...
1,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...
2,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...
3,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...
4,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...


Препроцессинг текста

In [7]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [13]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

for line in df['text']:
    spls = preprocess_txt(line)
    sentences.append(spls)

In [14]:
sentences = [i for i in sentences if len(i) > 2]

### Создание модели

Создание модели Word2Vec

In [15]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=3)

Создание модели FastText


In [16]:
modelFT = FastText(sentences=sentences, size=300, min_count=3, window=5, workers=8)

In [19]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

counter = 0
index_map = {}

for line in df['text']:
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line
    text = preprocess_txt(line)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in text:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [20]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

### Проверка работы модели

In [32]:
TEXT = "чемпионат мира по футболу"

In [33]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Главный тренер сборной России по тяжелой атлетике Александр Венков отметил хороший результат наших спортсменов на Универсиаде-2013, а также заявил, что соревнования в Казани были этапом подготовки к чемпионату мира.n«Я очень доволен результатом. Универсиаду мы рассматриваем как один из этапов подготовки к чемпионату мира, — сказал тренер. — Для тяжелоатлетов приоритетом будет чемпионат мира. Результаты нормальные, ребята отлично поработали. Впереди у нас достаточный период времени, чтобы восстановиться и хорошо подготовиться к чемпионату мира. У нас три целенаправленных сбора к чемпионату мира. Надеюсь, что ребята подойдут в хорошем состоянии».nСборная России завоевала в соревнованиях по тяжелой атлетике шесть золотых медалей.',
 'Президент Федерации фигурного катания на коньках России Александр Горшков рассказал о критериях отбора участников на чемпионат мира.nn«Участник чемпионата мира будет определен по итогам чемпионата Европы и финала Кубка России, который будет проходить в сере

In [34]:
get_response(TEXT, ft_index, modelFT, index_map)

['В Гармиш-Партенкирхене (Германия) проходит 16-й этап Кубка мира по горнолыжному спорту среди мужчин. В гигантском слаломе победил француз Алексис Пинтуро. Второе время показал австриец Марсель Хиршер, третье — чемпион мира в этой дисциплине американец Тед Лигети. Россиянин Степан Зуев не пробился во вторую попытку с 38-м временем.',
 'Россиянин Камиль Ибрагимов завоевал золотую медаль на чемпионате мира среди юниоров по фехтованию на сабле, который проходит в Порече (Хорватия).nВ финальном поединке Ибрагимов победил прошлогоднего обладателя юниорского Кубка мира венгра Андраша Сатмари — 15:14.nРоссиянка Лейла Пириева в финале первенства мира среди кадетов по фехтованию на рапире, который проходит также в Порече, уступила итальянке Камилле Манчини\xa0- 12:15 и стала серебряным призером турнира.',
 "В Валь д'Изере (Франция) проходит шестой этап Кубка мира по горнолыжному спорту среди женщин. В скоростном спуске победила швейцарка Лара Гут, вторая стала американка Леан Смит, третьей — е